In [1]:
import matplotlib.pyplot as plt
import math
import time
import functools


def timer(func):
    """Record elapsed time in a function"""
    @functools.wraps(func)
    def wrapper_timer(*args, **kwargs):
        t = time.time()
        func(*args, **kwargs)
        print("[%s] Elapsed %.6f second" % (func.__name__, time.time()-t))
        return func(*args, **kwargs)
    return wrapper_timer

In [2]:
def y_square(p, a, b, x):
    """Compute y^2 in form x^3+ax+b mod p
    """
    return (x**3+a*x+b) % p


def is_y_exist(p, a, b, x):
    """Check whether any y exists for given x in ECC
    Returns:
        -1 is not exists, a number otherwise
    """
    y_sqr = y_square(p, a, b, x)
    for i in range(p):
        if ((i*i) % p == y_sqr):
            return i
    return -1


def encode_char(character):
    """Encoding character to an integer"""
    return 10 + ord(character.lower())-ord('a')


def decode_code(code):
    """Encoding integer to a character"""
    return chr(ord('a') + code-10)


def generate_x(m, k, offset):
    """Yielding x by a function"""
    return m*k+offset


def encode(k, n, p, a, b, message_char):
    """Return encoded point"""
    m = encode_char(message_char)

    # Find solveable y
    for i in range(1, k):
        x = generate_x(m, k, i)
        y = is_y_exist(p, a, b, x)
        if (y != -1):
            break
    return (x, y)


def decode(k, x):
    """Return message char"""
    code = math.floor((x-1)/k)
    return decode_code(code)

In [3]:
@timer
def do_encoding(k, n, p, a, b, messages):
    """Do the encoding things"""
    enc_messages = [encode(k, n, p, a, b, m) for m in messages]
    return enc_messages


@timer
def do_decoding(k, message_point):
    """Do the decoding things"""
    dec_messages = [decode(k, point[0]) for point in message_point]
    return dec_messages

## Encoding in ECC using koblitz

In [8]:
# CONFIG
k = 20
p = 751
a = -1
b = 188
# TODO: figure out what 'n' did to the encoding-decoding stuffs
n = 727

# Samples
messages = "akulapar"

# Encoding
print("Encoded messages")
enc_messages = do_encoding(k, n, p, a, b, messages)
print(enc_messages)

# Decoding
print("\nDecoded messages")
dec_messages = do_decoding(k, enc_messages)
print("".join(dec_messages))

Encoded messages
[do_encoding] Elapsed 0.000896 second
[(201, 5), (401, 153), (603, 164), (421, 362), (201, 5), (501, 155), (201, 5), (543, 61)]

Decoded messages
[do_decoding] Elapsed 0.000015 second
akulapar
